In [9]:
import numpy as np
import pandas as pd
from collections import Counter
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
import fire

In [2]:
train_data = pd.read_csv('../input/train.csv')
test_data = pd.read_csv('../input/test.csv')
user_data = pd.read_csv('../input/user.csv', encoding='latin-1')
book_data = pd.read_csv('../input/book.csv', encoding='latin-1')

C:\Users\z\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
book_data.fillna('', inplace=True)
user_data.fillna('', inplace=True)

In [4]:
train_data.head()

,User-ID,ISBN,Book-Rating
0,8,0002005018,5
1,8,0374157065,0
2,8,0393045218,0
3,8,0399135782,0
4,8,0425176428,0


In [5]:
test_data.head()

,id,User-ID,ISBN
0,0,8,0060973129
1,1,8,0671870432
2,2,8,074322678X
3,3,8,0771074670
4,4,14,0689821166


In [6]:
user_data.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",
1,2,"stockton, california, usa",18
2,3,"moscow, yukon territory, russia",
3,4,"porto, v.n.gaia, portugal",17
4,5,"farnborough, hants, united kingdom",


In [7]:
book_data.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company


In [8]:
train_m1 = pd.merge(train_data, user_data, on=["User-ID"], how='left')
train_m2 = pd.merge(train_m1, book_data, on=["ISBN"], how='left')

test_m1 = pd.merge(test_data, user_data, on=["User-ID"], how='left')
test_m2 = pd.merge(test_m1, book_data, on=["ISBN"], how='left')

In [ ]:
def process2new(df_raw,train=True):
#     'User-ID', 'ISBN', 'Book-Rating', 'Location', 'Age', 'Book-Title',
#        'Book-Author', 'Year-Of-Publication', 'Publisher'
    loc0,loc1,loc2= list(zip(*map(lambda x:x.split(","), list(df_raw["Location"]))))
    loc0 = [x.strip() for x in loc0]
    loc1 = [x.strip() for x in loc1]
    loc2 = [x.strip() for x in loc2]
    
    User_id = list(df_raw["User-ID"])
    ISBN = list(df_raw["ISBN"])
    Age = [int(x) if isinstance(x, float) else 0 for x in list(df_raw["Age"])]
    Book_Title = list(df_raw["Book-Title"])
    Book_Author = list(df_raw["Book-Author"])
    Year_Of_Publication = list(df_raw["Year-Of-Publication"])
    Publisher = list(df_raw["Publisher"])
    
    res_d = {'User-ID':User_id, 'ISBN':ISBN, 'Loc0':loc0,'Loc1':loc1,"Loc2":loc2, 'Age':Age, 'Book-Title':Book_Title,
      'Book-Author':Book_Author, 'Year-Of-Publication':Year_Of_Publication, 'Publisher':Publisher}  
    if train:
        Book_Rating = df_raw["Book-Rating"]
        res_d['Book-Rating'] = Book_Rating
    pd_res = pd.DataFrame(res_d)    
    return res_d, pd_res 


_, train_m2 = process2new(train_m2)
_, test_m2 = process2new(test_m2, False)

In [ ]:
User_ID = list(train_m2["User-ID"]) + list(test_m2["User-ID"])
Loc0 = list(train_m2["Loc0"]) + list(test_m2["Loc0"])
Loc1 = list(train_m2["Loc1"]) + list(test_m2["Loc1"])
Loc2 = list(train_m2["Loc2"]) + list(test_m2["Loc2"])
Age = list(train_m2["Age"]) + list(test_m2["Age"])
Book_Title = list(train_m2["Book-Title"]) + list(test_m2["Book-Title"])
Book_Author = list(train_m2["Book-Author"]) + list(test_m2["Book-Author"])
Year_Of_Publication= list(train_m2["Year-Of-Publication"]) + list(test_m2["Year-Of-Publication"])
Publisher = list(train_m2["Publisher"]) + list(test_m2["Publisher"])

In [ ]:
def count2dict(li):
    d = {}
    lii = Counter(li).most_common()
    for (x,y) in  lii:
        if y < 3:
            continue
        d[x] = len(d)
    return d,lii 
def count2dictTitle(li):
    d = {}
    w = []
    for x in li:
        x = re.sub(",|\.|\?|\:|\!|\(|\)|\{|\}|\&|\*"," ",x)
        x = re.sub("\d"," ",x)
        x = x.split()
        w += x
    lii = Counter(w).most_common()
    res = []
    for (x,y) in lii:
        if y < 3:
            continue
        else:
            res.append((x,y))
            d[x] = len(d)
    return res,d
def count2dictLoc(li):
    d = {}
    w = []
    for x in li:
        w += x.split(",") 
    lii = Counter(w).most_common()
    for (x,y) in  lii:
        xx = x.strip()
        if xx is None or y < 3:
            continue
        d[x.strip()] = len(d)
    return d,lii 

In [ ]:
# Counter(Age)
# train_m2
UserDict,mcU = count2dict(User_ID)
l0Dict,_ = count2dict(Loc0)
l1Dict,_ = count2dict(Loc1)
l2Dict,_ = count2dict(Loc2)
Book_AuthorDict,mcB = count2dict(Book_Author)
Year_Of_PublicationDict = count2dict(Year_Of_Publication)
PublisherDict,mcP = count2dict(Publisher)
freqs,Book_TitleDict = count2dictTitle(Book_Title)

In [ ]:
def trans2id(df_raw,train=True):
    User_id = list(df_raw["User-ID"])
    ISBN = list(df_raw["ISBN"])
    Age = [int(x) for x in list(df_raw["Age"])]
    Book_Title = list(df_raw["Book-Title"])
    Book_Author = list(df_raw["Book-Author"])
    Year_Of_Publication = list(df_raw["Year-Of-Publication"])
    Publisher = list(df_raw["Publisher"])
    Loc0,Loc1,Loc2 = list(df_raw["Loc0"]),list(df_raw["Loc1"]),list(df_raw["Loc2"])
    
    def h(x,d):
        return [d[w]+1 if w in d else 0 for w in x]
    
    User_id_n = h(User_id,UserDict)
    loc0_n = h(Loc0,l0Dict) 
    loc1_n = h(Loc1,l1Dict) 
    loc2_n = h(Loc2,l2Dict) 
    
    Book_Author_n  = h(Book_Author,Book_AuthorDict)
    Year_Of_Publication_n = h(Year_Of_Publication,Year_Of_PublicationDict)
    Publisher_n = h(Publisher,PublisherDict)
    
    res_d = {'User-ID':User_id_n, 'ISBN':ISBN, 'Loc0':loc0_n,'Loc1':loc1_n,"Loc2":loc2_n, 'Age':Age, 'Book-Title':Book_Title,
      'Book-Author':Book_Author_n, 'Year-Of-Publication':Year_Of_Publication_n, 'Publisher':Publisher_n} 
    print(len(User_id_n),len(ISBN),len(loc0_n),len(loc1_n),len(loc2_n),len(Age),len(Book_Title),len(Book_Author_n),len(Year_Of_Publication_n),len(Publisher_n))
    if train:
        Book_Rating = df_raw["Book-Rating"]
        res_d['Book-Rating'] = Book_Rating
    pd_res = pd.DataFrame(res_d)    
    return pd_res,res_d

In [ ]:
id_train,_ = trans2id(train_m2)
id_test,_ = trans2id(test_m2,False)

In [ ]:
id_train.to_csv("./process_train.csv",index=None)
id_test.to_csv("./process_test.csv",index=None)